# <center> Module 3g - Multiple client connections for TCP sockets
## <center> SYSE 549: Secure Vehicle and Industrial Networking
## <center> <img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" /> 
### <center> Instructor: Dr. Jeremy Daily<br>Written By: Jerry Duggan

The connection oriented nature of TCP adds complexity for multiple client connections.  The server must simultaneously handle incoming connection establishment requests from new clients, while at the same time providing its service to existing clients.  One way to handle these concurrent operations is to use threads, as illustrated in this notebook.

Step 1 - Start a Python CLI window

Start a Python CLI window and connect to the directory in which you have extracted these notebooks.

Step 2 - Run the server program below

In [ ]:
import socket
from threading import Thread

HOST = '127.0.0.1'  # Standard loopback interface address (localhost)
PORT = 12354        # Port to listen on (non-privileged ports are > 1023)

class EchoThread(Thread):
    def __init__(self, conn):
        super().__init__()
        self.conn = conn

    def run(self):
        print()
        print(f"Echoing {self.conn}")
        numLines = 0
        while True:
            data = self.conn.recv(1024)
            if not data:
                print("end of data, stopping")
                return
            self.conn.sendall(data)
            numLines += 1
            print(f"received data from {self.conn}")
            print(f"  data: {data}, count: {numLines}")

def main(host, port):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((host, port))
    s.listen()

    while True:
        print("Main thread waiting for connections")
        srvConn, addr = s.accept()
        eThread = EchoThread(srvConn)
        print(f"connection from {addr}, spawning echo thread {eThread.name}")
        eThread.start()

main(HOST, PORT)

Main thread waiting for connections
connection from ('127.0.0.1', 52893), spawning echo thread Thread-5

Main thread waiting for connections
Echoing <socket.socket fd=1892, family=2, type=1, proto=0, laddr=('127.0.0.1', 12354), raddr=('127.0.0.1', 52893)>
received data from <socket.socket fd=1892, family=2, type=1, proto=0, laddr=('127.0.0.1', 12354), raddr=('127.0.0.1', 52893)>
  data: b'Hello, world (27556)', count: 1
received data from <socket.socket fd=1892, family=2, type=1, proto=0, laddr=('127.0.0.1', 12354), raddr=('127.0.0.1', 52893)>
  data: b'Hello, world (27556)', count: 2
received data from <socket.socket fd=1892, family=2, type=1, proto=0, laddr=('127.0.0.1', 12354), raddr=('127.0.0.1', 52893)>
  data: b'Hello, world (27556)', count: 3
received data from <socket.socket fd=1892, family=2, type=1, proto=0, laddr=('127.0.0.1', 12354), raddr=('127.0.0.1', 52893)>
  data: b'Hello, world (27556)', count: 4
received data from <socket.socket fd=1892, family=2, type=1, proto=0, la

Now run the 'TCPSimpleClient.py' program in your Python CLI window:

<span style="font-family:Courier;">python TCPSimpleClient.py</span>

This program uses threads to address both of the issues described in the overview about multiple client connections.  First, and most obvious, is that the program spawns an individual thread for each client, which reads the incoming data and sends it back to the client.  Because these operations are in separate threads, they don't interfere with each other, nor do they interfere with the main thread, which is responsible for handling incoming client connections.

Secondly, this program deals with the client context issue by using the Thread context to manage its client-specific data, in this case the line count (the numLines variable).  Doing this is acceptable for simple programs, but anything more complex than this should use an explicit data structure for client context.